In [28]:
import pandas as pd
import numpy as np
import sklearn.model_selection as ms
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as nr



%matplotlib inline

data = pd.read_json('https://raw.githubusercontent.com/wncc/learners-space/master/Machine%20Learning/MId%20Term%20Assignment/data.json')
data.head()



,0,1,2
0,-3.005382,-1.701191,1
1,-2.908105,-1.699589,1
2,-2.958954,-1.660940,1
3,-2.922840,-1.638722,1
4,-2.924240,-1.508256,1


In [29]:
onesandzeros = data.iloc[:,2].value_counts()
print(onesandzeros)



1    4679
0    1321
Name: 2, dtype: int64


In [30]:
nr.seed(0)

X = data.iloc[:,:2]
Y = data.iloc[:,2:]
x_train,x_test,y_train,y_test = ms.train_test_split(X,Y, test_size = 0.5)

In [31]:
np.array(x_train).reshape(3000,2)

array([[ 0.38110957,  1.31420618],
       [ 2.41908323,  1.40595449],
       [-0.86450147, -1.03095657],
       ...,
       [-1.35025675,  0.02093702],
       [-0.31304283, -1.52201001],
       [-0.38777214, -0.67340907]])

In [39]:
print(x_train)
print(y_train)


             0         1
3491  0.381110  1.314206
5495  2.419083  1.405954
2122 -0.864501 -1.030957
5379  2.267834  0.987245
2722 -0.333623 -0.976846
...        ...       ...
4931  1.876250 -0.568660
3264  0.164360  0.403848
1653 -1.350257  0.020937
2607 -0.313043 -1.522010
2732 -0.387772 -0.673409

[3000 rows x 2 columns]
      2
3491  0
5495  1
2122  0
5379  1
2722  0
...  ..
4931  1
3264  1
1653  1
2607  0
2732  0

[3000 rows x 1 columns]


In [32]:
A0 = np.array(pow(x_train[0],4)).reshape(1, x_train.shape[0])
A1 = np.array(pow(x_train[0],3)*x_train[1]).reshape(1, x_train.shape[0])
A2 = np.array(pow(x_train[0],2)*x_train[1]**2).reshape(1, x_train.shape[0])
A3 = np.array(pow(x_train[0],1)*x_train[1]**3).reshape(1, x_train.shape[0])
A4 = np.array(pow(x_train[0],4) ).reshape(1, x_train.shape[0])
A = np.vstack((A0, A1, A2, A3, A4))
A = A.T
print(A)
A.shape

[[ 2.10959661e-02  7.27466616e-02  2.50857284e-01  8.65048317e-01
   2.10959661e-02]
 [ 3.42454790e+01  1.99032362e+01  1.15676236e+01  6.72302305e+00
   3.42454790e+01]
 [ 5.58551146e-01  6.66097161e-01  7.94350581e-01  9.47298507e-01
   5.58551146e-01]
 ...
 [ 3.32403378e+00 -5.15423284e-02  7.99213183e-04 -1.23925661e-05
   3.32403378e+00]
 [ 9.60317954e-03  4.66905292e-02  2.27008722e-01  1.10371334e+00
   9.60317954e-03]
 [ 2.26103056e-02  3.92652879e-02  6.81885003e-02  1.18416847e-01
   2.26103056e-02]]


(3000, 5)

In [33]:
coef = np.array([0.001, 0.001, 0.001, 0.001, 0.001])

In [34]:
   y=y_train
for i in range (250):
    coef = coef.reshape(5,1) - 0.2*(np.dot(np.array((np.array(1/(1+
           (np.exp(-(np.dot(A,coef.reshape(5,1)))))))).reshape(y.shape[0],1) - y).reshape([1,y.shape[0]]), A)/y.shape[0]).reshape(5,1)
    
print(coef)
y_train=y

[[ 1.24137094]
 [-0.04987999]
 [-1.65999722]
 [ 0.00712061]
 [ 1.24137094]]


In [35]:
def h(A, coef):
  return np.array(1/(1+(np.exp(-f(A,coef)))) )
def f(A, coef):
  fun = np.dot(A,coef.reshape(5,1))
  return fun
def l(y, A, coef): 
  return (-np.dot(y.T, np.log(h(A, coef))) - np.dot((1-y).T ,np.log(1-h(A, coef))) )/y.shape[0]
def g(A, coef, y):
  return np.dot(np.array(h(A, coef).reshape(y.shape[0],1) - y).reshape([1,y.shape[0]]), A)/y.shape[0]
def gradesc(A, y, coef,L):       # here coef is column vector 
  for i in range (250):
    coef = coef.reshape(5,1) - L*g(A, coef, y).reshape(5,1)
  return coef 

def predict(x_test, coef = gradesc(A,y_train, coef, 0.001) ):
  A0 = np.array(x_test[0]**4).reshape(1, x_test.shape[0])
  A1 = np.array(x_test[0]**3 * x_test[1]).reshape(1, x_test.shape[0])
  A2 = np.array(x_test[0]**2 * x_test[1]**2).reshape(1, x_test.shape[0])
  A3 = np.array(x_test[0]**1 * x_test[1]**3).reshape(1, x_test.shape[0])
  A4 = np.array(x_test[1]**4 ).reshape(1, x_test.shape[0])
  A_ = np.vstack((A0, A1, A2, A3, A4))
  A_ = A_.T
  print('Mean Loss :' ,np.sum(l(y_test,A_,coef)))
  return h(A_, coef)

probs = predict(x_test)
print(probs)

Mean Loss : nan
[[0.9442101 ]
 [0.97706775]
 [1.        ]
 ...
 [0.94166624]
 [0.88208165]
 [1.        ]]


<ipython-input-35-3b7cc90285c7>:7: RuntimeWarning: divide by zero encountered in log
  return (-np.dot(y.T, np.log(h(A, coef))) - np.dot((1-y).T ,np.log(1-h(A, coef))) )/y.shape[0]


In [36]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:]])
y_pred = score_model(probs, 0.5035)
pred_df = pd.DataFrame(y_pred)
print('Predicted values')
print(pred_df.value_counts())

Predicted values
1    2923
0      77
dtype: int64


In [37]:
print('Actual values')
y_test.value_counts()

Actual values


2
1    2346
0     654
dtype: int64

In [38]:
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[1,1] + '             %5d' % conf[1,0])
    print('Actual negative    %6d' % conf[0,1] + '             %5d' % conf[0,0])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][1] + '        %6d' % metrics[3][0])
    print('Precision  %6.2f' % metrics[0][1] + '        %6.2f' % metrics[0][0])
    print('Recall     %6.2f' % metrics[1][1] + '        %6.2f' % metrics[1][0])
    print('F1         %6.2f' % metrics[2][1] + '        %6.2f' % metrics[2][0])


    
print_metrics(y_test, y_pred)    
print_metrics(y_test, y_pred)

                 Confusion matrix
                 Score positive    Score negative
Actual positive      2305                41
Actual negative       618                36

Accuracy  0.78
 
           Positive      Negative
Num case     2346           654
Precision    0.79          0.47
Recall       0.98          0.06
F1           0.87          0.10
                 Confusion matrix
                 Score positive    Score negative
Actual positive      2305                41
Actual negative       618                36

Accuracy  0.78
 
           Positive      Negative
Num case     2346           654
Precision    0.79          0.47
Recall       0.98          0.06
F1           0.87          0.10
